## Prepare QA

In [2]:
# ...existing code...
import json
def prepare_qa(file_path: str):
    """
    Prepares QA pairs from a JSONL file.
    """
    with open(file_path, 'r', encoding='utf-8') as file:  # Specify UTF-8 encoding
        data = [json.loads(line) for line in file]

    qa = []
    for item in data:
        qa.append({'question': item['question'], 'options': item['options'], 'answer': item['answer_idx']})
    
    return qa

# ...existing code...

In [3]:
QA = prepare_qa(r"E:\Git_clone\RAG\qa_dataset\data_clean\questions\US\4_options\phrases_no_exclude_test.jsonl")

## Evaluation

In [4]:
import chatbot, importlib
importlib.reload(chatbot)
from chatbot import Chatbot
chatbot = Chatbot("mistral")

### Vector retriever

In [5]:
from vectordb import create_retriever

In [6]:
vretriever = create_retriever(r"/workspaces/YuE/faiss_index")

### Graph retriever

In [7]:
from graphdb import gretriever

In [8]:
def retrieve(query, rag_type = None, k = 5):
    if rag_type == None:
        return ""
    elif rag_type == "rag":
        contexts = vretriever.get_relevant_documents(query, k=k)
        return "\n\n".join([context.page_content for context in contexts])
    elif rag_type == "grag":
        contexts = vretriever.get_relevant_documents(query, k=k)
        contexts = gretriever("\n".join([context.page_content for context in contexts]), extract_model="mistral")
        return "\n".join([context for context in contexts])

In [9]:
def process_qa(qa, rag_type = None, k = 5):
    context = retrieve(qa['question'], rag_type, k=k)
    prompt = f"""
    You are a medical expert. Answer the question by coorperate the provided context with your knowledgement.
    Document: {context}
    Question: {qa['question']}
    Options:
    A: {qa['options']['A']}
    B: {qa['options']['B']}
    C: {qa['options']['C']}
    D: {qa['options']['D']}
    Answer: (Only return A:, B:, C:, or D: without any explanation or other text. Do not include the context or the question in your answer.) 
    """
    response = chatbot.chat(prompt)
    # print("Prompt: ", prompt, '\n')
    if "A:" in response:
        answer = "A"
    elif "B:" in response:
        answer = "B"
    elif "C:" in response:
        answer = "C"
    elif "D:" in response:
        answer = "D"
    else:
        answer = None

    # print(response," ", answer, "  ", qa['answer'])
    return answer == qa['answer']

In [10]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time

def process_qa_with_retry(qa, rag_type=None, retries=8, delay=1):
    """
    Processes a single QA pair with retry logic for rate-limiting errors.

    Parameters:
        qa (dict): A single QA pair to process.
        rag_type (str): Type of retriever to use (e.g., "rag", "grag").
        retries (int): Number of retries for rate-limiting errors.
        delay (int): Delay in seconds between retries.

    Returns:
        bool: Whether the processed answer matches the expected answer.
    """
    for attempt in range(retries):
        try:
            return process_qa(qa, rag_type)
        except Exception as e:
            if "rate limit" in str(e).lower():
                # print(f"Rate limit encountered. Retrying in {delay} seconds... (Attempt {attempt + 1}/{retries})")
                time.sleep(delay)
            else:
                print(f"Error processing QA: {e}")
                break
    return False  # Return False if all retries fail

def process_qa_parallel(QA, n_workers=4, rag_type=None, k=5):
    """
    Processes QA pairs in parallel and calculates accuracy.

    Parameters:
        QA (list): List of QA pairs to process.
        n_workers (int): Number of worker threads to use.
        rag_type (str): Type of retriever to use (e.g., "rag", "grag").

    Returns:
        float: Accuracy of the processed QA pairs.
    """
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        # Use tqdm for progress tracking
        results = list(tqdm(
            executor.map(lambda qa: process_qa_with_retry(qa, rag_type, k), QA),
            total=len(QA),
            desc="Processing QA"
        ))

    # Calculate and return accuracy
    total_correct = sum(results)
    accuracy = total_correct / len(QA)
    return accuracy

In [ ]:
accuracy = process_qa_parallel(QA, n_workers=1, rag_type="grag", k=8)
print(f"Accuracy: {accuracy}")


C:\Users\This PC\AppData\Local\Temp\ipykernel_6284\3995853207.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  contexts = vretriever.get_relevant_documents(query, k=k)
Processing QA:   0%|          | 0/1273 [00:00<?, ?it/s]

Processing QA:   1%|          | 14/1273 [15:31<30:18:04, 86.64s/it]

2025-04-29 14:54:44,393 - camel.models.model_manager - ERROR - Error processing with model: <camel.models.mistral_model.MistralModel object at 0x000001F358D7E910>
2025-04-29 14:54:44,396 - camel.agents.chat_agent - ERROR - An error occurred while running model mistral-large-latest, index: 0
Traceback (most recent call last):
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\agents\chat_agent.py", line 777, in _get_model_response
    response = self.model_backend.run(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\models\model_manager.py", line 226, in run
    raise exc
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\models\model_manager.py", line 216, in run
    response = self.current_model.run(messages, response_format, tools)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\This PC\App

Processing QA:   3%|▎         | 39/1273 [43:42<20:28:13, 59.72s/it]

2025-04-29 15:22:56,390 - camel.models.model_manager - ERROR - Error processing with model: <camel.models.mistral_model.MistralModel object at 0x000001F2BE17D7D0>
2025-04-29 15:22:56,390 - camel.agents.chat_agent - ERROR - An error occurred while running model mistral-large-latest, index: 0
Traceback (most recent call last):
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\agents\chat_agent.py", line 777, in _get_model_response
    response = self.model_backend.run(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\models\model_manager.py", line 226, in run
    raise exc
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\models\model_manager.py", line 216, in run
    response = self.current_model.run(messages, response_format, tools)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\This PC\App

...

Processing QA:   7%|▋         | 94/1273 [1:48:21<16:39:43, 50.88s/it]

2025-04-29 16:27:33,871 - camel.models.model_manager - ERROR - Error processing with model: <camel.models.mistral_model.MistralModel object at 0x000001F2B73E94D0>
2025-04-29 16:27:33,872 - camel.agents.chat_agent - ERROR - An error occurred while running model mistral-large-latest, index: 0
Traceback (most recent call last):
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\agents\chat_agent.py", line 777, in _get_model_response
    response = self.model_backend.run(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\models\model_manager.py", line 226, in run
    raise exc
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\models\model_manager.py", line 216, in run
    response = self.current_model.run(messages, response_format, tools)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\This PC\App

Processing QA:   9%|▊         | 109/1273 [2:01:06<17:38:35, 54.57s/it]

2025-04-29 16:40:19,440 - camel.models.model_manager - ERROR - Error processing with model: <camel.models.mistral_model.MistralModel object at 0x000001F2E6283CD0>
2025-04-29 16:40:19,440 - camel.agents.chat_agent - ERROR - An error occurred while running model mistral-large-latest, index: 0
Traceback (most recent call last):
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\agents\chat_agent.py", line 777, in _get_model_response
    response = self.model_backend.run(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\models\model_manager.py", line 226, in run
    raise exc
  File "c:\Users\This PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\camel\models\model_manager.py", line 216, in run
    response = self.current_model.run(messages, response_format, tools)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\This PC\App

Processing QA:   9%|▊         | 110/1273 [2:03:43<27:30:09, 85.13s/it]

In [ ]:
accuracy

0.7250589159465829